### CNN for Feature Extraction 
* image/video - 
e.g. pinterest(hashtag as labeling-supervise learning)
e.g. Adobe DeepFont (to detect specific designed font)

* paper review youtube channel:
https://www.youtube.com/@user-ow3gm9zd1b/featured


* CNN (mostly used in image classification)
* Fully-connected layer (어디 위치에 쓰이느냐에 따라 이름이 달라진다)

## Terminology
- **channel**: A channel is like a separate color or information in an image. In color images, there are different channels for red, green, and blue. Each channel holds specific color information.

- **Filter**: A filter is like a special tool used to modify an image. It can do things like making an image blurry, enhancing edges, or finding specific patterns. The filter uses a set of numbers to determine how it changes the image. 이미지에서 필요한/주요한 특징값만 추출하고 필요없는 특징값은 버리는 역할. CNN에서 feature값을 보고 판단/예측할 수 있다. filter안의 값들 = weight 값. *filter의 개수는 많을수록 좋고, 깊이도 깊을수록 더 좋다.* 중심부에서 더 많은 feature를 받게 된다.

- Stride: How far to go to the right/bottom to perform the next convolution (중앙값 뿐만 아니라 다른 곳에도 가중치를 두기 위해서, 즉 몇칸씩 이동할지)

- **Feature map**: A feature map is like a map that highlights specific features in an image. It's created by applying filters to the image. Each feature map shows the presence or intensity of a particular feature, such as edges or textures.

- **padding**: Padding is like adding extra space around an image. It helps to keep the image size consistent during certain operations. Padding can be done by adding extra pixels with specific values, like zeros, around the edges of the image.

- zero padding (0으로 테두리):  It involves adding extra rows and columns of zeros around the edges of an image or feature map before applying convolutional operations. 중앙필터 방지, & 가중으로 필터를 씌워야할 필요가 있을 때 - 이미지의 작아짐 방지. 

### CNN composed layers
- Convolutional Layer
- Fully Connected Layer
- Pooling Layer (cleaning up 정리정돈) - Max Pooling, Average Pooling. 짧은 시간안에 목표까지 도달할 수 있다.



## Text Classification
CNN: 영상 이미지 뿐만 아니라 모든 matrix data 가능, including text data
test -> 의미를 가지고 있는 vector값으로 표현...


### <추세>  
~2019 회귀 MLP. 이미지 CNN -> MLP MIXER , 자연어/시계열 RNN  -> CNN-> CNN + RNN

2021~
image, video - transformer (자연어에서 사용하던 transformer가 영상쪽에서 사용되자 기존 CNN 모두 이겨버림! 최고 성능을 내는 것은 결국 transformer이다)
BUT transformer (CNN보다 훨씬 더 많은 cost) - 1%~2% +성능이기 때문에 실무에서는 더 작은 model인 CNN 선호.

-> CNN + transformer (기반 model - 엄청난 성능, 그러나 엄청나게 크다) - 더 큰 transformer 모델이 무조건 성능이 좋기 때문에 성능적 개선을 연구하는 움직임은 침체. - 이제는 data적인 부분 (labeled/unlabeled data) 등 실무적인 부분에 대한 연구로 전향되고 있다. 

### MNIST dataset
- CNN사용 유무에 따른 연산량의 차이, 속도 차이 비교/연구

- Difference in computational complexity and speed comparison/research depending on the use of CNN.

- MNIST data: 다양한 case, data class의 정확성, data가 많고 다양하다. 실무에서 좋은 data 확보하는 것이 매우 중요하다!

In [26]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [27]:
mnist = tf.keras.datasets.mnist

In [28]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [29]:
# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0
# X_train, X_test norm(0-1)

### try training without CNN

In [30]:
model = tf.keras.models.Sequential()
#  multidimensional feature maps are reshaped into a single vector,(Here, it converts the 2D image into a 1D vector of size 784(28*28))
model.add(tf.keras.layers.Flatten(input_shape=(28, 28) )) # input shape: data하나의 크기가 관건이다. 
model.add(tf.keras.layers.Dense(128, activation='relu')) 
# input is 1D vector of size 784, each units in Dense layer has a weight for each input feature, 784*128=100,352 weights, 128 bias terms,
# Thus, 100,352+128=100,480 parameters

model.add(tf.keras.layers.Dropout(0.2)) # 특정 feature들을 가리고 학습하는 것. to prevent overfitting. 

model.add(tf.keras.layers.Dense(10 , activation ='softmax')) # This layer is defined with 10 units and uses the softmax activation function.
# input is 1D vector of size 128. Thus, 10*128 = 1,280 weights. & bias term of 10, 
# Thus, 1,280 + 10 = 1,290 parameters

model.summary() 
# Total: 100,480 + 1,290 = 101,770 parameters in the model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Building loss function and optimizer

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, epochs = 5, 
          validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 11s 3ms/step - loss: 0.2970 - accuracy: 0.9146 - val_loss: 0.1405 - val_accuracy: 0.9592
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1453 - accuracy: 0.9565 - val_loss: 0.0990 - val_accuracy: 0.9695
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1088 - accuracy: 0.9671 - val_loss: 0.0857 - val_accuracy: 0.9740
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0897 - accuracy: 0.9723 - val_loss: 0.0814 - val_accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0781 - accuracy: 0.9754 - val_loss: 0.0765 - val_accuracy: 0.9765
